In [213]:
import pandas as pd
# we can see all of the columns
pd.set_option('max_columns', None)
import os

import re

In [214]:
# first load in the NHS EXCEL Dataset, this might be harder
df = pd.read_excel('hospital_sets/hosp-epis-stat-admi-prov-leve-2008-09-tab.xls', sheet_name='Table E_0809')

In [215]:
df.shape

(479, 21)

In [62]:
need_to_see = pd.read_excel('hospital_sets/hosp-epis-stat-admi-prov-leve-2010-11-tab.xls', sheet_name='Table E_1011')

In [63]:
need_to_see.head().append(need_to_see.tail())

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Table E: Finished admission episodes by primar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,NaN,Responsible statistician:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
526,NaN,"Chris Dew, HES/SUS Analysis (Service) Section ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [92]:
attempt_2 = pd.read_csv('hospital_sets/csv/hosp-epis-stat-admi-prov-leve-2019-20-tab.csv', skiprows=6).rename(columns={'Unnamed: 0':'provider_code'})

# Load in the nhs Admission dataset


In [97]:
attempt_2.shape
print('hosp-epis-stat-admi-prov-leve-2019-20-tab.csv'[32:37].replace('-', ''))
# rename(columns='Unnamed: 0':'region_or_provider_code')


1920


In [233]:
# Horrifically hacky - iterate through the CSV files and load them in, os.walk is much better than os.listdir

directory= 'hospital_sets/csv'

df_lst = []

# new - with csv files 
for filename in os.listdir(directory):
    if filename.endswith('csv'):
        date_range =filename[32:37].replace('-', '')

        if date_range in ['0809','0910']:
            # changed encoding -- stackoverflow to solve unicodedecodeerror, https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
            tmp_df = pd.read_csv(os.path.join(directory, filename), encoding='ISO-8859-1')
            
            tmp_df.columns = tmp_df.columns.str.strip().str.replace('\n', '')
            tmp_df = tmp_df.rename(columns=lambda x: re.sub(r'\s\W\w\d\d-\w\d\d\W$', '' ,x))
            tmp_df = tmp_df.rename(columns=lambda x: re.sub(r'\W\w\d\d-\w\d\d\W$', '' ,x))



            
            
            tmp_df.rename(columns={
                                  'ï»¿ SHA/Provider':'Commissioning Region/Provider',
                                  'SHA/Provider':'Commissioning Region/Provider',
                                  'Area Team/Provider':'Commissioning Region/Provider',
                                  }, inplace=True)
            
            
            print(list(tmp_df.columns))

            
            tmp_df = tmp_df.loc[:, :'Factors influencing health status and contact with health services']
            
            tmp_df['Date Period'] = date_range

            print('checks\n')
            print('THE YEAR TO CHECK IS', filename[32:37].replace('-', ''))
            print(tmp_df.shape)
            print(list(tmp_df.columns))
            print('\n')

            df_lst.append(tmp_df)
        elif date_range == '1011':
            # changed encoding -- stackoverflow to solve unicodedecodeerror, https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
            tmp_df = pd.read_csv(os.path.join(directory, filename), skiprows=10, encoding='ISO-8859-1')
            
            tmp_df.columns = tmp_df.columns.str.strip()
            tmp_df.columns = tmp_df.columns.str.replace('\n', '')
            tmp_df = tmp_df.rename(columns=lambda x: re.sub(r'\s\W\w\d\d-\w\d\d\W$', '' ,x))
            tmp_df = tmp_df.rename(columns=lambda x: re.sub(r'\W\w\d\d-\w\d\d\W$', '' ,x))





            
            tmp_df.rename(columns={
#                                   'Factors influencing health status and contact with health services':'Factors influencing health status and contact with health services(Z00-Z99)',
                                  'ï»¿ SHA/Provider':'Commissioning Region/Provider',
                                  'SHA/Provider':'Commissioning Region/Provider',
                                  'Area Team/Provider':'Commissioning Region/Provider',
#                                   'Factors influencing health status and contact with health services \n(Z00-Z99)':'Factors influencing health status and contact with health services(Z00-Z99)',
                                  }, inplace=True)
            
            print(list(tmp_df.columns))
        
            tmp_df = tmp_df.loc[:, :'Factors influencing health status and contact with health services']
            
            tmp_df['Date Period'] = date_range

            
            print('checks\n')
            print('THE YEAR TO CHECK IS', filename[32:37].replace('-', ''))
            print(tmp_df.shape)
            print('\n')

            df_lst.append(tmp_df)
        else:
            # changed encoding -- stackoverflow to solve unicodedecodeerror, https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
            tmp_df = pd.read_csv(os.path.join(directory, filename), skiprows=6, encoding='ISO-8859-1')
            
            tmp_df.columns = tmp_df.columns.str.strip()
            tmp_df.columns = tmp_df.columns.str.replace('\n', '')
#             tmp_df.columns = tmp_df.columns.str.replace(r'\s\W\w\d\d-\w\d\d\W$', r'\W\w\d\d-\w\d\d\W$')
            tmp_df = tmp_df.rename(columns=lambda x: re.sub(r'\s\W\w\d\d-\w\d\d\W$', '' ,x))
            tmp_df = tmp_df.rename(columns=lambda x: re.sub(r'\W\w\d\d-\w\d\d\W$', '' ,x))


            
            tmp_df.rename(columns={
                                  'ï»¿ SHA/Provider':'Commissioning Region/Provider',
                                  'SHA/Provider':'Commissioning Region/Provider',
                                  'Area Team/Provider':'Commissioning Region/Provider',
                                  }, inplace=True)
            
            print(list(tmp_df.columns))

            tmp_df = tmp_df.loc[:, :'Factors influencing health status and contact with health services']
            tmp_df['Date Period'] = date_range

            print('checks\n')
            print('THE YEAR TO CHECK IS', filename[32:37].replace('-', ''))

            print(tmp_df.shape)
            print(list(tmp_df.columns))
            print('\n')

            df_lst.append(tmp_df)

['Commissioning Region/Provider', 'Certain infectious and parasitic diseases', 'Neoplasms', 'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism', 'Endocrine, nutritional and metabolic diseases', 'Mental and behavioural disorders', 'Diseases of the nervous system', 'Diseases of the eye and adnexa', 'Diseases of the ear and mastoid process', 'Diseases of the circulatory system', 'Diseases of the respiratory system', 'Diseases of the digestive system', 'Diseases of the skin and subcutaneous tissue', 'Diseases of the musculoskeletal system and connective tissue', 'Diseases of the genitourinary system', 'Pregnancy, childbirth and the puerperium', 'Certain conditions originating in the perinatal period', 'Congenital malformations, deformations and chromosomal abnormalities', 'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified', 'Injury, poisoning and certain other consequences of external causes', 'Factors 

['Provider Code', 'ONS Geography Code', 'Commissioning Region/Provider', 'Certain infectious and parasitic diseases', 'Neoplasms', 'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism', 'Endocrine, nutritional and metabolic diseases', 'Mental and behavioural disorders', 'Diseases of the nervous system', 'Diseases of the eye and adnexa', 'Diseases of the ear and mastoid process', 'Diseases of the circulatory system', 'Diseases of the respiratory system', 'Diseases of the digestive system', 'Diseases of the skin and subcutaneous tissue', 'Diseases of the musculoskeletal system and connective tissue', 'Diseases of the genitourinary system', 'Pregnancy, childbirth and the puerperium', 'Certain conditions originating in the perinatal period', 'Congenital malformations, deformations and chromosomal abnormalities', 'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified', 'Injury, poisoning and certain other cons

1516

In [227]:
# tmp_df = tmp_df.rename(columns={'Factors influencing health status and contact with health services':'Factors influencing health status and contact with health services (Z00-Z99)'})

# len(df_lst)
# for df in df_lst:
#     print(df.shape)

df_lst = []
print(df_lst)

[]


In [234]:
nhs_hospital_adm2 = pd.concat(df_lst, axis=0, ignore_index=True)

In [235]:
nhs_hospital_adm2.shape

(6301, 26)

In [236]:
nhs_hospital_adm2.head(50)

,Commissioning Region/Provider,Certain infectious and parasitic diseases,Neoplasms,Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism,"Endocrine, nutritional and metabolic diseases",Mental and behavioural disorders,Diseases of the nervous system,Diseases of the eye and adnexa,Diseases of the ear and mastoid process,Diseases of the circulatory system,Diseases of the respiratory system,Diseases of the digestive system,Diseases of the skin and subcutaneous tissue,Diseases of the musculoskeletal system and connective tissue,Diseases of the genitourinary system,"Pregnancy, childbirth and the puerperium",Certain conditions originating in the perinatal period,"Congenital malformations, deformations and chromosomal abnormalities","Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified","Injury, poisoning and certain other consequences of external causes",Factors influencing health status and contact with health services,Date Period,Unnamed: 0,Codes for special purposes,Provider Code,ONS Geography Code
0,ENGLAND,"172,683","1,524,205","235,309","201,988","164,982","294,641","549,545","89,407","926,461","757,003","1,589,433","289,435","1,077,760","880,349","1,349,505","199,543","110,997","1,698,401","914,260","1,126,785",0809,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0809,NaN,NaN,NaN,NaN
2,Q33 East Midlands Strategic Health Authority,"12,060","135,824","17,969","16,033","11,855","27,063","44,247","6,646","63,339","54,101","107,213","20,383","82,496","63,587","106,872","13,421","6,747","129,310","66,542","89,982",0809,NaN,NaN,NaN,NaN
3,5N6 Derbyshire County PCT,53,535,78,63,284,387,794,*,655,402,594,394,"1,701",620,-,-,*,"1,185",760,"1,577",0809,NaN,NaN,NaN,NaN
4,5N8 Nottinghamshire County Teaching PCT,19,301,*,32,24,216,-,-,117,104,36,46,103,75,-,-,*,242,315,49,0809,NaN,NaN,NaN,NaN
5,5PA Leicestershire County and Rutland PCT,69,"1,017",97,68,58,417,"2,021",15,615,216,"1,920",629,"1,124","1,062",254,-,18,"1,102",956,687,0809,NaN,NaN,NaN,NaN
6,5PC Leicester City PCT,-,*,-,*,-,*,-,-,*,11,*,*,7,*,-,-,-,297,9,-,0809,NaN,NaN,NaN,NaN
7,5PD Northamptonshire Teaching PCT,*,254,*,*,*,281,-,*,72,33,8,*,22,*,-,-,*,674,62,34,0809,NaN,NaN,NaN,NaN
8,"NT213 Nuffield Health, Derby Hospital",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,*,-,-,0809,NaN,NaN,NaN,NaN
9,"NT226 Nuffield Health, Leicester Hospital",-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0809,NaN,NaN,NaN,NaN


In [237]:
if not os.path.exists('hospital_sets/csv/concat'):
    os.makedirs('hospital_sets/csv/concat')
    
nhs_hospital_adm2.to_csv('hospital_sets/csv/concat/nhs_hospital_admissions_2008_2020.CSV', encoding='utf-8')

# Load in the nhs mortality dataset


In [210]:
mortality_directory = 'mortality_hospital'

df_lst = []

# cooler way to get all file names from folder
# https://stackoverflow.com/questions/19932130/iterate-through-folders-then-subfolders-and-print-filenames-with-path-to-text-f
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
            for name in files:
                    if 'SHMI diagnosis group breakdown' in name and '.csv' in name:
                        r.append(os.path.join(root, name))
    return r

def create_list_df(lst):
    dff_lst = []
    # iterate through, create df and apply tolist
    for filename in lst:
        tm_df = pd.read_csv(filename)
        
        date_p = filename[36:47]
        
        tm_df['Date Period'] = date_p
        
        tm_df = tm_df.rename(columns={
            'DIAGNOSIS_GROUP_NUMBER': 'DIAGNOSIS GROUP',
            'PROVIDER': 'PROVIDER_CODE',
            'DENOMINATOR': 'SPELLS',
            'DIAGNOSIS_GROUP': 'DIAGNOSIS GROUP'
                                     })
        dff_lst.append(tm_df)
    return dff_lst
        


In [211]:
# get the list
list_of_csv = list_files('mortality_hospital')
# print(list_of_csv[0])
# print('\n')
# for item in list_of_csv:
#     print(item[36:47])


# get list of dfs
dfss = create_list_df(list_of_csv)
print(len(dfss))

# concat the dataframes
df_concc = pd.concat(dfss, axis=0, ignore_index=True)
df_concc.head()

29


,INDICATOR_CODE,DIAGNOSIS GROUP,PROVIDER_CODE,PROVIDER_NAME,SPELLS,OBSERVED,EXPECTED,Date Period,DIAGNOSIS_GROUP_DESCRIPTION,SHMI_VALUE,SHMI_BANDING,PO_LL,PO_UL,OD_LL,OD_UL
0,I00699,1,R1H,BARTS HEALTH NHS TRUST,248,*,9.4433,Apr13-Mar14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I00699,1,RA2,ROYAL SURREY COUNTY HOSPITAL NHS FOUNDATION TRUST,*,0,0.0267,Apr13-Mar14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I00699,1,RA3,WESTON AREA HEALTH NHS TRUST,*,0,0.04,Apr13-Mar14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,I00699,1,RA4,YEOVIL DISTRICT HOSPITAL NHS FOUNDATION TRUST,*,0,0.0102,Apr13-Mar14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,I00699,1,RA7,UNIVERSITY HOSPITALS BRISTOL NHS FOUNDATION TRUST,17,0,0.9597,Apr13-Mar14,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
if not os.path.exists('mortality_hospital/concat_final'):
    os.makedirs('mortality_hospital/concat_final')
    
df_concc.shape

df_concc.to_csv('mortality_hospital/concat_final/hospital_mortality_rate.csv', encoding='utf-8')
